# Data Query and Access

## Purpose

This notebook is **Step 0** of the AM-QADF workflow: Query raw point cloud data from the MongoDB data warehouse. You'll query each data source separately, as each source will be mapped to its own voxel grid in subsequent steps.

**Workflow Position**: This is the first step before grid creation and signal mapping.

## Learning Objectives

By the end of this notebook, you will:
- ✅ Connect to MongoDB using the infrastructure layer
- ✅ Query point cloud data from each source separately (laser, CT, ISPM, hatching)
- ✅ Apply spatial and temporal filters interactively
- ✅ Understand data structure and format for each source
- ✅ Prepare data for mapping to separate grids (Notebook 04)

## Estimated Duration

45-60 minutes

---

## Overview

This notebook queries **raw point cloud data** from multiple sources. Each source is queried **separately** because:

1. **Each source gets its own grid** (created in Notebook 03)
2. **Each source is mapped independently** (in Notebook 04)
3. **Each source is corrected independently** (in Notebook 05)
4. **Grids are fused together** (in Notebook 06)

### Data Sources

Query point cloud data from:
- 🛤️ **Hatching Paths**: Layer and path data (for Grid D)
- ⚡ **Laser Parameters**: Power, speed, energy density (for Grid A)
- 🔬 **CT Scans**: Defect locations and density (for Grid B)
- 🌡️ **ISPM Monitoring**: Temperature and sensor measurements (for Grid C)
- 📐 **STL Models**: 3D model geometry (for bounding box determination)

### Workflow Context

**Current Step (Notebook 01)**: Query point cloud data per source
- ↓
**Next Step (Notebook 03)**: Create separate empty grids per source
- ↓
**Then (Notebook 04)**: Map queried point cloud data to respective grids
- ↓
**Then (Notebook 02)**: Align grids temporally and spatially
- ↓
**Then (Notebook 05)**: Correct and calibrate each grid independently
- ↓
**Finally (Notebook 06)**: Fuse all corrected grids into one unified grid

### Key Points

- **Query each source separately** - Don't combine them yet
- **Store results per source** - Each source will have its own grid
- **Point cloud format** - Data will be mapped to voxel grids in Notebook 04
- **Spatial/Temporal filters** - Apply filters to reduce data size if needed

Use the interactive widgets below to query data from each source - no coding required!

In [1]:
# Setup: Import required libraries
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Add parent directory and src directory to path for imports
notebook_dir = Path().resolve()
project_root = notebook_dir.parent
src_dir = project_root / 'src'

# Add project root to path (for src.infrastructure imports)
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Add src directory to path (for am_qadf imports)
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Core imports
import ipywidgets as widgets
from ipywidgets import (
    VBox, HBox, Accordion, Tab, Dropdown, RadioButtons, 
    Checkbox, Button, Output, Text, IntSlider, FloatSlider,
    Layout, Box, Label, FloatText, IntText,
    HTML as WidgetHTML
)
from IPython.display import display, Markdown, HTML, clear_output
import pandas as pd
import numpy as np
from datetime import datetime
import time

# Load environment variables from development.env
import os
env_file = project_root / 'development.env'
if env_file.exists():
    with open(env_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line and not line.startswith('#') and '=' in line:
                key, value = line.split('=', 1)
                value = value.strip('"\'')
                os.environ[key] = value
    print("✅ Environment variables loaded from development.env")

# Try to import AM-QADF and infrastructure
INFRASTRUCTURE_AVAILABLE = False
QUERY_AVAILABLE = False
STL_CLIENT_AVAILABLE = False

try:
    from src.infrastructure.database import get_connection_manager
    INFRASTRUCTURE_AVAILABLE = True
except (ImportError, TypeError, Exception) as e:
    INFRASTRUCTURE_AVAILABLE = False
    print(f"⚠️ Infrastructure layer not available: {type(e).__name__}: {e}")

try:
    from am_qadf.query import UnifiedQueryClient, STLModelClient
    from am_qadf.query.base_query_client import QueryResult, SpatialQuery
    QUERY_AVAILABLE = True
    STL_CLIENT_AVAILABLE = True
except ImportError as e:
    QUERY_AVAILABLE = False
    STL_CLIENT_AVAILABLE = False
    print(f"⚠️ Query clients not available: {e}")
    QueryResult = None  # Fallback for demo mode
    SpatialQuery = None  # Fallback for demo mode

MATPLOTLIB_AVAILABLE = False
PYVISTA_AVAILABLE = False
PYSLM_AVAILABLE = False

# Visualization imports
try:
    import matplotlib.pyplot as plt
    MATPLOTLIB_AVAILABLE = True
except ImportError:
    MATPLOTLIB_AVAILABLE = False
    print("⚠️ Matplotlib not available")

try:
    import pyvista as pv
    pv.set_jupyter_backend('static')  # Set backend for Jupyter
    PYVISTA_AVAILABLE = True
except ImportError:
    PYVISTA_AVAILABLE = False
    print("⚠️ PyVista not available")

try:
    import pyslm
    PYSLM_AVAILABLE = True
except ImportError:
    PYSLM_AVAILABLE = False
    print("⚠️ pyslm not available")

# Initialize MongoDB connection
mongo_client = None
unified_client = None
stl_client = None

if INFRASTRUCTURE_AVAILABLE:
    try:
        manager = get_connection_manager(env_name="development")
        mongo_client = manager.get_mongodb_client()
        
        # Test connection
        if mongo_client and mongo_client.is_connected():
            print("✅ MongoDB connection established")
            
            # Test authentication by trying to list collections
            try:
                test_collection = mongo_client.get_collection('stl_models')
                count = test_collection.count_documents({})
                print(f"✅ Authentication verified (found {count} STL models)")
            except Exception as e:
                print(f"⚠️ Authentication test failed: {e}")
                print("   This may indicate authentication issues")
            
            # Initialize query clients
            if QUERY_AVAILABLE:
                try:
                    unified_client = UnifiedQueryClient(mongo_client=mongo_client)
                    print("✅ UnifiedQueryClient initialized")
                except Exception as e:
                    print(f"⚠️ Error initializing UnifiedQueryClient: {e}")
                    import traceback
                    traceback.print_exc()
            
            if STL_CLIENT_AVAILABLE:
                try:
                    stl_client = STLModelClient(mongo_client=mongo_client)
                    print("✅ STLModelClient initialized")
                except Exception as e:
                    print(f"⚠️ Error initializing STLModelClient: {e}")
                    import traceback
                    traceback.print_exc()
        else:
            print("⚠️ MongoDB client not connected")
            print("   Using demo mode with synthetic data")
    except Exception as e:
        print(f"⚠️ MongoDB connection failed: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        print("   Using demo mode with synthetic data")
else:
    print("⚠️ Using demo mode - infrastructure layer unavailable")

print("✅ Setup complete!")


✅ Environment variables loaded from development.env
✅ MongoDB connection established
✅ Authentication verified (found 3 STL models)
✅ UnifiedQueryClient initialized
✅ STLModelClient initialized
✅ Setup complete!


## Interactive Data Query Interface

Use the widgets below to query data from the warehouse. Select a model, choose data sources, apply filters, and view results - all interactively!


In [2]:
# Create Interactive Query Interface

# Global state
current_model_id = None
current_model_info = None
query_results = {}
query_dataframes = {}

# ============================================
# Top Panel: Model Selection and Actions
# ============================================

# Get available models
models = []
model_options = [("━━━ Choose a model ━━━", None)]

if stl_client and mongo_client:
    try:
        models = stl_client.list_models(limit=100)
        model_options.extend([
            (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", 
             m.get('model_id')) for m in models
        ])
        if len(model_options) == 1:
            model_options.append(("No models available", None))
    except Exception as e:
        print(f"⚠️ Error loading models: {e}")
        model_options.append(("Error loading models", None))
else:
    # Demo mode: create synthetic model options
    model_options.extend([
        ("Demo Model 1 (demo-001)", "demo-001"),
        ("Demo Model 2 (demo-002)", "demo-002"),
        ("Demo Model 3 (demo-003)", "demo-003")
    ])

model_dropdown = Dropdown(
    options=model_options,
    value=None,
    description='Model:',
    style={'description_width': 'initial'},
    layout=Layout(width='400px')
)

query_button = Button(
    description='Query',
    button_style='success',
    icon='search',
    layout=Layout(width='100px')
)

clear_button = Button(
    description='Clear',
    button_style='',
    icon='trash',
    layout=Layout(width='100px')
)

top_panel = HBox([
    model_dropdown,
    query_button,
    clear_button
], layout=Layout(justify_content='flex-start', padding='10px', border='1px solid #ccc'))

# ============================================
# Left Panel: Query Configuration
# ============================================

# Data Source Selection
# Note: get_all_data() queries ALL sources by default, so all checkboxes are True by default
data_source_label = widgets.HTML("<b>Data Sources:</b>")
stl_checkbox = Checkbox(value=True, description='STL', style={'description_width': 'initial'}, layout=Layout(width='auto'))
hatching_checkbox = Checkbox(value=True, description='Hatching', style={'description_width': 'initial'}, layout=Layout(width='auto'))
laser_checkbox = Checkbox(value=True, description='Laser', style={'description_width': 'initial'}, layout=Layout(width='auto'))
ct_checkbox = Checkbox(value=True, description='CT', style={'description_width': 'initial'}, layout=Layout(width='auto'))
ispm_checkbox = Checkbox(value=True, description='ISPM', style={'description_width': 'initial'}, layout=Layout(width='auto'))

data_sources = VBox([
    data_source_label,
    HBox([stl_checkbox, hatching_checkbox]),
    HBox([laser_checkbox, ct_checkbox, ispm_checkbox])
], layout=Layout(padding='5px'))

# Spatial Filter Section
spatial_label = widgets.HTML("<b>Spatial Filter:</b>")
spatial_mode = RadioButtons(
    options=[('Full Model', 'full'), ('Custom Bounding Box', 'custom'), ('Interactive Selection', 'interactive')],
    value='full',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Custom bounding box sliders
bbox_x_min = FloatSlider(value=-50.0, min=-1000.0, max=1000.0, step=1.0, description='X Min:', style={'description_width': 'initial'})
bbox_x_max = FloatSlider(value=50.0, min=-1000.0, max=1000.0, step=1.0, description='X Max:', style={'description_width': 'initial'})
bbox_y_min = FloatSlider(value=-50.0, min=-1000.0, max=1000.0, step=1.0, description='Y Min:', style={'description_width': 'initial'})
bbox_y_max = FloatSlider(value=50.0, min=-1000.0, max=1000.0, step=1.0, description='Y Max:', style={'description_width': 'initial'})
bbox_z_min = FloatSlider(value=0.0, min=-1000.0, max=1000.0, step=1.0, description='Z Min:', style={'description_width': 'initial'})
bbox_z_max = FloatSlider(value=100.0, min=-1000.0, max=1000.0, step=1.0, description='Z Max:', style={'description_width': 'initial'})

bbox_sliders = VBox([
    bbox_x_min, bbox_x_max,
    bbox_y_min, bbox_y_max,
    bbox_z_min, bbox_z_max
], layout=Layout(display='none'))  # Hidden by default

def update_spatial_controls(change):
    """Show/hide bounding box sliders based on mode."""
    if change['new'] == 'custom':
        bbox_sliders.layout.display = 'flex'
    else:
        bbox_sliders.layout.display = 'none'

spatial_mode.observe(update_spatial_controls, names='value')

spatial_section = VBox([
    spatial_label,
    spatial_mode,
    bbox_sliders
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Temporal Filter Section
temporal_label = widgets.HTML("<b>Temporal Filter:</b>")
temporal_mode = RadioButtons(
    options=[('All', 'all'), ('Layer-based', 'layer'), ('Time-based', 'time')],
    value='all',
    description='Mode:',
    style={'description_width': 'initial'}
)

# Layer range sliders
layer_min = IntSlider(value=0, min=0, max=1000, step=1, description='Layer Min:', style={'description_width': 'initial'})
layer_max = IntSlider(value=100, min=0, max=1000, step=1, description='Layer Max:', style={'description_width': 'initial'})

layer_sliders = VBox([
    layer_min, layer_max
], layout=Layout(display='none'))

# Time range inputs
time_min = FloatText(value=0.0, description='Time Min (s):', style={'description_width': 'initial'})
time_max = FloatText(value=1000.0, description='Time Max (s):', style={'description_width': 'initial'})

time_inputs = VBox([
    time_min, time_max
], layout=Layout(display='none'))

def update_temporal_controls(change):
    """Show/hide temporal controls based on mode."""
    if change['new'] == 'layer':
        layer_sliders.layout.display = 'flex'
        time_inputs.layout.display = 'none'
    elif change['new'] == 'time':
        layer_sliders.layout.display = 'none'
        time_inputs.layout.display = 'flex'
    else:
        layer_sliders.layout.display = 'none'
        time_inputs.layout.display = 'none'

temporal_mode.observe(update_temporal_controls, names='value')

temporal_section = VBox([
    temporal_label,
    temporal_mode,
    layer_sliders,
    time_inputs
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Parameter Filter Section (Collapsible)
parameter_label = widgets.HTML("<b>Parameter Filters:</b>")
parameter_expand = Checkbox(value=False, description='Show Parameter Filters', style={'description_width': 'initial'})

# Laser parameter filters
laser_power_min = FloatSlider(value=0.0, min=0.0, max=1000.0, step=10.0, description='Power Min (W):', style={'description_width': 'initial'})
laser_power_max = FloatSlider(value=1000.0, min=0.0, max=1000.0, step=10.0, description='Power Max (W):', style={'description_width': 'initial'})
scan_speed_min = FloatSlider(value=0.0, min=0.0, max=5000.0, step=50.0, description='Speed Min (mm/s):', style={'description_width': 'initial'})
scan_speed_max = FloatSlider(value=5000.0, min=0.0, max=5000.0, step=50.0, description='Speed Max (mm/s):', style={'description_width': 'initial'})

parameter_sliders = VBox([
    laser_power_min, laser_power_max,
    scan_speed_min, scan_speed_max
], layout=Layout(display='none'))

def update_parameter_controls(change):
    """Show/hide parameter sliders."""
    if change['new']:
        parameter_sliders.layout.display = 'flex'
    else:
        parameter_sliders.layout.display = 'none'

parameter_expand.observe(update_parameter_controls, names='value')

parameter_section = VBox([
    parameter_label,
    parameter_expand,
    parameter_sliders
], layout=Layout(padding='5px', border='1px solid #ddd'))

# Left panel assembly
left_panel = VBox([
    data_sources,
    spatial_section,
    temporal_section,
    parameter_section
], layout=Layout(width='300px', padding='10px', border='1px solid #ccc'))

# ============================================
# Center Panel: Results Display
# ============================================

# Results tabs
results_output = Output(layout=Layout(height='600px', overflow='auto'))

# Create tab widgets for different views
table_output = Output()
stats_output = Output()
viz_output = Output()
# Create separate output widgets for 2x2 grid visualization
hatching_viz_output = Output(layout=Layout(height='300px', border='1px solid #ccc'))
laser_viz_output = Output(layout=Layout(height='300px', border='1px solid #ccc'))
ispm_viz_output = Output(layout=Layout(height='300px', border='1px solid #ccc'))
ct_viz_output = Output(layout=Layout(height='300px', border='1px solid #ccc'))

# Create 2x2 grid layout
viz_grid = VBox([
    HBox([hatching_viz_output, laser_viz_output]),
    HBox([ispm_viz_output, ct_viz_output])
], layout=Layout(padding='5px'))

# Keep viz_output for backward compatibility, but use viz_grid
# viz_output = VBox([
#     widgets.HTML("<h4>Visualization Grid</h4>"),
#     viz_grid
# ])

export_output = Output()

results_tabs = Tab(children=[table_output, stats_output, viz_output, export_output])
results_tabs.set_title(0, 'Table')
results_tabs.set_title(1, 'Statistics')
results_tabs.set_title(2, 'Visualization')
results_tabs.set_title(3, 'Export')

center_panel = VBox([
    widgets.HTML("<h3>Query Results</h3>"),
    results_tabs
], layout=Layout(flex='1 1 auto', padding='10px', border='1px solid #ccc'))

# ============================================
# Right Panel: Quick Actions and Status
# ============================================

# Quick actions
quick_actions_label = widgets.HTML("<b>Quick Actions:</b>")
refresh_button = Button(description='Refresh Models', button_style='info', icon='refresh', layout=Layout(width='150px'))
export_button = Button(description='Export Results', button_style='', icon='download', layout=Layout(width='150px'))

quick_actions = VBox([
    quick_actions_label,
    refresh_button,
    export_button
], layout=Layout(padding='5px'))

# Status display
status_label = widgets.HTML("<b>Status:</b>")
status_text = widgets.HTML("Ready to query")
status_display = VBox([
    status_label,
    status_text
], layout=Layout(padding='5px'))

# Results summary
summary_label = widgets.HTML("<b>Results Summary:</b>")
summary_text = widgets.HTML("No query executed yet")
summary_display = VBox([
    summary_label,
    summary_text
], layout=Layout(padding='5px'))

right_panel = VBox([
    quick_actions,
    status_display,
    summary_display
], layout=Layout(width='250px', padding='10px', border='1px solid #ccc'))

# ============================================
# Bottom Panel: Progress and Status with Logging
# ============================================

# Status display widget
current_operation = WidgetHTML(value='<b>Status:</b> Ready to query')

# Progress bar
progress_bar = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Progress:',
    bar_style='info',
    style={'bar_color': '#1f77b4'},
    layout=Layout(width='100%')
)

# Query logs output
query_logs = Output(layout=Layout(height='200px', border='1px solid #ccc', overflow_y='auto'))

# Initialize logs
with query_logs:
    display(HTML("<p><i>Query logs will appear here...</i></p>"))

# Bottom status bar (shows Status | Progress | Time)
bottom_status = WidgetHTML(value='<b>Status:</b> Ready | <b>Progress:</b> 0% | <b>Time:</b> 0:00')
bottom_progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description='Overall:',
    bar_style='info',
    layout=Layout(width='100%')
)

# Error display (kept for backward compatibility)
error_display = widgets.HTML("", layout=Layout(padding='5px'))

# Enhanced bottom panel
bottom_panel = VBox([
    current_operation,
    progress_bar,
    WidgetHTML("<b>Query Logs:</b>"),
    query_logs,
    WidgetHTML("<hr>"),
    bottom_status,
    bottom_progress,
    error_display
], layout=Layout(padding='10px', border='1px solid #ccc'))

# Global time tracking
operation_start_time = None

# ============================================
# Logging Functions
# ============================================

def log_message(message: str, level: str = 'info'):
    """Log a message to the query logs with timestamp and emoji."""
    timestamp = datetime.now().strftime('%H:%M:%S')
    icons = {'info': 'ℹ️', 'success': '✅', 'warning': '⚠️', 'error': '❌'}
    icon = icons.get(level, 'ℹ️')
    with query_logs:
        print(f"[{timestamp}] {icon} {message}")

def update_status(operation: str, progress: int = None):
    """Update the status display and progress."""
    global operation_start_time
    current_operation.value = f'<b>Status:</b> {operation}'
    if progress is not None:
        progress_bar.value = progress
        bottom_progress.value = progress
        if operation_start_time:
            elapsed = time.time() - operation_start_time
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> {time.strftime("%M:%S", time.gmtime(elapsed))}'
        else:
            bottom_status.value = f'<b>Status:</b> {operation} | <b>Progress:</b> {progress}% | <b>Time:</b> 0:00'

# ============================================
# Query Execution Function
# ============================================

def execute_query(button):
    """Execute query with current widget settings."""
    global current_model_id, query_results, query_dataframes, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with query_logs:
        clear_output(wait=True)
    
    log_message("Starting query operation...", 'info')
    update_status("Initializing query...", 0)
    status_text.value = "<span style='color: blue;'>Executing query...</span>"
    error_display.value = ""
    
    # Get model ID
    model_id = model_dropdown.value
    if not model_id:
        log_message("Please select a model", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select a model</span>"
        status_text.value = "<span style='color: red;'>Error: No model selected</span>"
        update_status("Please select a model", 0)
        return
    
    current_model_id = model_id
    log_message(f"Querying model: {model_id[:8]}...", 'info')
    update_status(f"Querying model {model_id[:8]}...", 20)
    
    # Get selected data sources
    selected_sources = []
    if stl_checkbox.value:
        selected_sources.append('stl')
    if hatching_checkbox.value:
        selected_sources.append('hatching')
    if laser_checkbox.value:
        selected_sources.append('laser')
    if ct_checkbox.value:
        selected_sources.append('ct')
    if ispm_checkbox.value:
        selected_sources.append('ispm')
    
    if not selected_sources:
        log_message("Please select at least one data source", 'warning')
        error_display.value = "<span style='color: red;'>⚠️ Please select at least one data source</span>"
        status_text.value = "<span style='color: red;'>Error: No data sources selected</span>"
        update_status("Please select data sources", 0)
        return
    
    log_message(f"Selected data sources: {', '.join(selected_sources)}", 'info')
    update_status("Preparing query filters...", 40)
    
    # Get spatial filter
    spatial_bbox = None
    if spatial_mode.value == 'custom':
        spatial_bbox = (
            (bbox_x_min.value, bbox_y_min.value, bbox_z_min.value),
            (bbox_x_max.value, bbox_y_max.value, bbox_z_max.value)
        )
        log_message(f"Spatial filter: Custom bounding box {spatial_bbox}", 'info')
    else:
        log_message("Spatial filter: Full model", 'info')
    
    update_status("Applying filters...", 60)
    
    # Get temporal filter
    temporal_range = None
    if temporal_mode.value == 'layer':
        # Convert layer range to time (simplified - would need layer-time mapping)
        temporal_range = (layer_min.value * 10.0, layer_max.value * 10.0)  # Assume 10s per layer
        log_message(f"Temporal filter: Layers {layer_min.value}-{layer_max.value} (time: {temporal_range[0]:.1f}-{temporal_range[1]:.1f}s)", 'info')
    elif temporal_mode.value == 'time':
        temporal_range = (time_min.value, time_max.value)
        log_message(f"Temporal filter: Time range {temporal_range[0]:.1f}-{temporal_range[1]:.1f}s", 'info')
    else:
        log_message("Temporal filter: All", 'info')
    
    update_status("Executing query...", 80)
    
    # Execute query
    try:
        if unified_client and mongo_client:
            log_message("Querying MongoDB for data...", 'info')
            # Real query - get all data with error handling
            result = {
                'model_id': model_id,
                'stl_model': None,
                'hatching_layers': None,
                'laser_parameters': None,
                'ct_scan': None,
                'ispm_monitoring': None
            }
            
            # Get STL model
            if 'stl' in selected_sources:
                try:
                    log_message("Querying STL model...", 'info')
                    if unified_client.stl_client:
                        result['stl_model'] = unified_client.stl_client.get_model(model_id)
                        log_message("STL model retrieved", 'success')
                except Exception as e:
                    log_message(f"Error querying STL model: {str(e)}", 'error')
                    result['stl_model'] = {'error': str(e)}
            
            # Get hatching layers
            if 'hatching' in selected_sources:
                try:
                    log_message("Querying hatching layers...", 'info')
                    if unified_client.hatching_client:
                        layers = unified_client.hatching_client.get_layers(model_id)
                        result['hatching_layers'] = layers if layers else []  # Ensure it's a list, not None
                        log_message(f"Hatching layers retrieved: {len(layers) if layers else 0} layers", 'success')
                except Exception as e:
                    log_message(f"Error querying hatching layers: {str(e)}", 'error')
                    result['hatching_layers'] = {'error': str(e)}
            
            # Get laser parameters
            if 'laser' in selected_sources:
                try:
                    log_message("Querying laser parameters...", 'info')
                    if unified_client.laser_client:
                        spatial_query = SpatialQuery(component_id=model_id)
                        result['laser_parameters'] = unified_client.laser_client.query(spatial=spatial_query)
                        log_message("Laser parameters retrieved", 'success')
                except Exception as e:
                    log_message(f"Error querying laser parameters: {str(e)}", 'error')
                    result['laser_parameters'] = {'error': str(e)}
            
            # Get CT scan data
            if 'ct' in selected_sources:
                try:
                    log_message("Querying CT scan data...", 'info')
                    if unified_client.ct_client:
                        ct_data = unified_client.ct_client.get_scan(model_id)
                        result['ct_scan'] = ct_data  # Can be None if not found
                        log_message("CT scan data retrieved", 'success')
                except Exception as e:
                    log_message(f"Error querying CT scan: {str(e)}", 'error')
                    result['ct_scan'] = {'error': str(e)}
            
            # Get ISPM monitoring data
            if 'ispm' in selected_sources:
                try:
                    log_message("Querying ISPM monitoring data...", 'info')
                    if unified_client.ispm_client:
                        spatial_query = SpatialQuery(component_id=model_id)
                        result['ispm_monitoring'] = unified_client.ispm_client.query(spatial=spatial_query)
                        log_message("ISPM monitoring data retrieved", 'success')
                except Exception as e:
                    log_message(f"Error querying ISPM monitoring: {str(e)}", 'error')
                    result['ispm_monitoring'] = {'error': str(e)}
            
            query_results = result
        else:
            log_message("Using demo mode (MongoDB not available)", 'info')
            # Demo mode: create synthetic results
            result = {
                'model_id': model_id,
                'stl_model': {'name': f'Model {model_id}', 'vertices': 1000} if 'stl' in selected_sources else None,
                'hatching_layers': [{'layer': i, 'paths': 10} for i in range(10)] if 'hatching' in selected_sources else None,
                'laser_parameters': {'count': 500, 'power_range': (100, 300)} if 'laser' in selected_sources else None,
                'ct_scan': {'defects': 5, 'density_range': (0.8, 1.0)} if 'ct' in selected_sources else None,
                'ispm_monitoring': {'samples': 1000, 'temp_range': (500, 1200)} if 'ispm' in selected_sources else None
            }
            query_results = result
            log_message("Demo data generated", 'success')
        
        update_status("Processing results...", 90)
        
        # Update results display
        update_results_display()


        
        # Update summary
        # Helper function to safely get count from result data
        def get_count(data):
            """Get count from data, handling both dict and QueryResult objects."""
            if data is None:
                return 0
            if QueryResult and isinstance(data, QueryResult):
                # QueryResult has points attribute
                return len(data.points) if hasattr(data, 'points') and data.points else 0
            elif isinstance(data, dict):
                # Dictionary - try common count keys
                return data.get('count', data.get('samples', len(data) if isinstance(data, (list, dict)) else 0))
            elif isinstance(data, list):
                return len(data)
            else:
                return 1 if data else 0
        
        total_items = sum([
            get_count(result.get('stl_model')),
            get_count(result.get('hatching_layers')),
            get_count(result.get('laser_parameters')),
            get_count(result.get('ct_scan')),
            get_count(result.get('ispm_monitoring'))
        ])
        
        log_message(f"Query completed: {total_items} total items from {len(selected_sources)} source(s)", 'success')
        
        summary_text.value = f"""
        <p><b>Model:</b> {model_id[:8]}...</p>
        <p><b>Sources:</b> {', '.join(selected_sources)}</p>
        <p><b>Total Items:</b> {total_items}</p>
        """
        
        # Calculate total execution time
        if operation_start_time:
            total_time = time.time() - operation_start_time
            log_message(f"Query operation completed in {total_time:.2f}s", 'success')
        else:
            log_message("Query operation completed successfully", 'success')
        
        # Update status
        status_text.value = "<span style='color: green;'>✅ Query completed successfully</span>"
        update_status("Query completed", 100)
        
    except Exception as e:
        log_message(f"Error during query: {str(e)}", 'error')
        import traceback
        log_message(f"Traceback: {traceback.format_exc()}", 'error')
        error_display.value = f"<span style='color: red;'>❌ Error: {str(e)}</span>"
        status_text.value = f"<span style='color: red;'>Error: {str(e)}</span>"
        update_status("Error during query", 0)

def update_results_display():
    """Update the results display tabs."""
    global query_results
    
    # Clear all outputs
    with table_output:
        clear_output(wait=True)
        if query_results:
            # Create a simple table display
            html = "<h4>Query Results</h4><table border='1' style='border-collapse: collapse; width: 100%;'>"
            html += "<tr><th>Source</th><th>Status</th><th>Details</th></tr>"
            
            for source, data in query_results.items():
                if source == 'model_id':
                    continue
                # Check if data exists and is not empty
                if data is not None:
                    if QueryResult and isinstance(data, QueryResult):
                        # Handle QueryResult objects
                        point_count = len(data.points) if hasattr(data, 'points') and data.points else 0
                        signal_count = len(data.signals) if hasattr(data, 'signals') and data.signals else 0
                        if point_count > 0 or signal_count > 0:
                            html += f"<tr><td>{source}</td><td style='color: green;'>✓</td><td>{point_count} points, {signal_count} signals</td></tr>"
                        else:
                            html += f"<tr><td>{source}</td><td style='color: orange;'>⚠</td><td>Query returned 0 points, {signal_count} signals (no data found)</td></tr>"
                    elif isinstance(data, dict):
                        if 'error' in data:
                            html += f"<tr><td>{source}</td><td style='color: red;'>Error</td><td>{data['error']}</td></tr>"
                        else:
                            details = ', '.join([f"{k}: {v}" for k, v in list(data.items())[:3]])
                            html += f"<tr><td>{source}</td><td style='color: green;'>✓</td><td>{details}</td></tr>"
                    elif isinstance(data, list):
                        if len(data) > 0:
                            html += f"<tr><td>{source}</td><td style='color: green;'>✓</td><td>{len(data)} items</td></tr>"
                        else:
                            html += f"<tr><td>{source}</td><td style='color: orange;'>⚠</td><td>Empty list (0 items found)</td></tr>"
                    else:
                        html += f"<tr><td>{source}</td><td style='color: green;'>✓</td><td>Available</td></tr>"
                else:
                    # get_all_data() queries all sources, so None means no data found for this source
                    html += f"<tr><td>{source}</td><td style='color: gray;'>-</td><td>No data available (query returned None)</td></tr>"
            
            html += "</table>"
            display(HTML(html))
        else:
            display(HTML("<p>No results to display. Execute a query first.</p>"))
    
    with stats_output:
        clear_output(wait=True)
        if query_results:
            html = "<h4>Statistics</h4><ul>"
            for source, data in query_results.items():
                if source == 'model_id':
                    continue
                if data:
                    if QueryResult and isinstance(data, QueryResult):
                        # Handle QueryResult objects
                        point_count = len(data.points) if hasattr(data, 'points') and data.points else 0
                        signal_count = len(data.signals) if hasattr(data, 'signals') else 0
                        html += f"<li><b>{source}:</b> {point_count} points, {signal_count} signals</li>"
                    elif isinstance(data, list):
                        html += f"<li><b>{source}:</b> {len(data)} items</li>"
                    elif isinstance(data, dict):
                        if 'count' in data:
                            html += f"<li><b>{source}:</b> {data['count']} records</li>"
                        elif 'error' in data:
                            html += f"<li><b>{source}:</b> Error - {data['error']}</li>"
                        else:
                            html += f"<li><b>{source}:</b> Available</li>"
                    else:
                        html += f"<li><b>{source}:</b> Available</li>"
            html += "</ul>"
            display(HTML(html))
        else:
            display(HTML("<p>No statistics available.</p>"))
    
    with viz_output:
        clear_output(wait=True)
        display(HTML("<h4>Visualization Grid</h4>"))
        display(viz_grid)

        # Check what data is available and show appropriate visualizations
        has_hatching = query_results.get('hatching_layers') is not None
        has_laser = query_results.get('laser_parameters') is not None
        has_ispm = query_results.get('ispm_monitoring') is not None
        has_ct = query_results.get('ct_scan') is not None

        if not any([has_hatching, has_laser, has_ispm, has_ct]):
            display(HTML("<p style='color:orange;'>⚠️ No data to visualize. Execute a query first.</p>"))
        else:
            display(HTML("<p>Select visualization type:</p>"))
    
            # Create visualization buttons
            viz_buttons = HBox([
                Button(description='Hatching', button_style='info', layout=Layout(width='120px')) if has_hatching else widgets.HTML(""),
                Button(description='Laser', button_style='info', layout=Layout(width='120px')) if has_laser else widgets.HTML(""),
                Button(description='ISPM', button_style='info', layout=Layout(width='120px')) if has_ispm else widgets.HTML(""),
                Button(description='CT Scan', button_style='info', layout=Layout(width='120px')) if has_ct else widgets.HTML(""),
            ])
    
            # Attach button handlers
            if has_hatching:
                viz_buttons.children[0].on_click(lambda b: visualize_hatching_with_pyslm())
            if has_laser:
                idx = 1 if has_hatching else 0
                viz_buttons.children[idx].on_click(lambda b: visualize_laser_parameters())
            if has_ispm:
                idx = sum([has_hatching, has_laser])
                viz_buttons.children[idx].on_click(lambda b: visualize_ispm_data())
            if has_ct:
                idx = sum([has_hatching, has_laser, has_ispm])
                viz_buttons.children[idx].on_click(lambda b: visualize_ct_scan())
    
            display(viz_buttons)
    
            # Auto-visualize first available data type
            if has_hatching:
                visualize_hatching_with_pyslm()
            if has_laser:
                visualize_laser_parameters()
            if has_ispm:
                visualize_ispm_data()
            if has_ct:
                visualize_ct_scan()
    with export_output:
        clear_output(wait=True)
        if query_results:
            display(HTML("<h4>Export Options</h4><p>Export functionality will be available in future versions.</p>"))
        else:
            display(HTML("<p>No data to export.</p>"))

def visualize_hatching_with_pyslm():
    """Visualize hatching data using custom HatchingVisualizer (inspired by pyslm)."""
    with hatching_viz_output:
        clear_output(wait=True)
        
        hatching_data = query_results.get('hatching_layers')
        stl_model = query_results.get('stl_model')
        
        if not hatching_data or len(hatching_data) == 0:
            display(HTML("<p style='color:orange;'>⚠️ No hatching data. Query hatching data first.</p>"))
            return
        
        display(HTML("<p>🔄 Generating hatching visualization...</p>"))
        
        try:
            from am_qadf.visualization.hatching_visualizer import HatchingVisualizer
            
            visualizer = HatchingVisualizer()
            
            # Option 1: Use MongoDB data directly (reconstructed)
            sample_layers = hatching_data[:min(10, len(hatching_data))]  # Sample first 10 layers
            
            fig, ax = visualizer.plot_layers(
                layers=sample_layers,  # MongoDB documents
                plot_contours=True,
                plot_hatches=True,
                plot_points=False,
                plot_3d=True,
                plot_colorbar=True,
                index='laser_power',  # Color by laser power (from MongoDB)
                colormap='plasma',
                linewidth=0.5
            )
            
            ax.set_title(f'Hatching Visualization ({len(sample_layers)} layers)')
            ax.set_xlabel('X (mm)')
            ax.set_ylabel('Y (mm)')
            ax.set_zlabel('Z (mm)')
            
            plt.tight_layout()
            plt.show()
            
            display(HTML(f"<p>✅ Visualized {len(sample_layers)} layers</p>"))
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
            

def visualize_laser_parameters():
    """Visualize laser parameters with color mapping."""
    with laser_viz_output:
        clear_output(wait=True)
        
        laser_data = query_results.get('laser_parameters')
        
        if not laser_data:
            display(HTML("<p style='color:orange;'>⚠️ No laser parameter data. Query laser data first.</p>"))
            return
        
        display(HTML("<p>🔄 Generating laser parameters visualization...</p>"))
        
        try:
            import pyvista as pv
            pv.set_jupyter_backend('static')
            
            from am_qadf.query.base_query_client import QueryResult
            if isinstance(laser_data, QueryResult):
                if not laser_data.points or len(laser_data.points) == 0:
                    display(HTML("<p style='color:orange;'>⚠️ No laser parameter points</p>"))
                    return
                
                max_points = 10000
                points_array = np.array(laser_data.points)
                if len(points_array) > max_points:
                    indices = np.random.choice(len(points_array), max_points, replace=False)
                    points = points_array[indices]
                else:
                    points = points_array
                    indices = np.arange(len(points))
                
                signals = laser_data.signals or {}
                if 'laser_power' in signals:
                    power_array = np.array(signals['laser_power'])
                    power = power_array[indices] if len(power_array) > max_points else power_array
                else:
                    power = np.ones(len(points)) * 250.0
                
                if points.shape[1] == 2:
                    z_coords = np.zeros((points.shape[0], 1))
                    points = np.hstack([points, z_coords])
                
                point_cloud = pv.PolyData(points)
                point_cloud['power'] = power
                
                plotter = pv.Plotter(notebook=True)
                plotter.add_mesh(
                    point_cloud,
                    scalars='power',
                    cmap='plasma',
                    point_size=5,
                    show_scalar_bar=True,
                    scalar_bar_args={'title': 'Laser Power (W)', 'n_labels': 5}
                )
                plotter.add_text(f'Laser Parameters Visualization ({len(points):,} points)', font_size=12)
                plotter.add_axes()
                plotter.show(jupyter_backend='static')
            else:
                display(HTML("<p style='color:orange;'>⚠️ Unexpected data format</p>"))
                
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

def visualize_ispm_data():
    """Visualize ISPM data with 2D plots."""
    with ispm_viz_output:
        clear_output(wait=True)
        
        ispm_data = query_results.get('ispm_monitoring')
        
        if not ispm_data:
            display(HTML("<p style='color:orange;'>⚠️ No ISPM data. Query ISPM data first.</p>"))
            return
        
        display(HTML("<p>🔄 Generating ISPM data visualization...</p>"))
        
        try:
            from am_qadf.query.base_query_client import QueryResult
            if isinstance(ispm_data, QueryResult):
                if not ispm_data.points or len(ispm_data.points) == 0:
                    display(HTML("<p style='color:orange;'>⚠️ No ISPM points</p>"))
                    return
                
                max_points = 5000
                points_array = np.array(ispm_data.points)
                if len(points_array) > max_points:
                    indices = np.random.choice(len(points_array), max_points, replace=False)
                    points = points_array[indices]
                else:
                    points = points_array
                    indices = np.arange(len(points))
                
                signals = ispm_data.signals or {}
                if 'melt_pool_temperature' in signals:
                    temps_array = np.array(signals['melt_pool_temperature'])
                    temps = temps_array[indices] if len(temps_array) > max_points else temps_array
                else:
                    temps = np.ones(len(points)) * 1700.0
                
                if 'melt_pool_size' in signals:
                    sizes_array = np.array(signals['melt_pool_size'])
                    sizes = sizes_array[indices] if len(sizes_array) > max_points else sizes_array
                else:
                    sizes = np.ones(len(points)) * 0.5
                
                if points.shape[1] >= 2:
                    coords = points[:, :2]
                    if points.shape[1] >= 3:
                        z_coords = points[:, 2]
                        z_min = float(np.min(z_coords))
                        z_max = float(np.max(z_coords))
                        z_mean = float(np.mean(z_coords))
                        z_info = f"Z: {z_min:.2f} to {z_max:.2f} mm (mean: {z_mean:.2f} mm)"
                    else:
                        z_info = "Z: N/A (2D data)"
                else:
                    coords = np.array([[p[0], p[1] if len(p) > 1 else 0] for p in points])
                    z_info = "Z: N/A"
                
                fig, axes = plt.subplots(2, 2, figsize=(14, 10))
                
                axes[0, 0].plot(temps[:1000], 'b-', alpha=0.7)
                axes[0, 0].set_xlabel('Sample Index')
                axes[0, 0].set_ylabel('Temperature (K)')
                axes[0, 0].set_title('Melt Pool Temperature Time Series')
                axes[0, 0].grid(True, alpha=0.3)
                
                axes[0, 1].hist(sizes, bins=50, color='green', alpha=0.7)
                axes[0, 1].set_xlabel('Melt Pool Size (mm)')
                axes[0, 1].set_ylabel('Frequency')
                axes[0, 1].set_title('Melt Pool Size Distribution')
                axes[0, 1].grid(True, alpha=0.3)
                
                if len(coords) > 0:
                    scatter = axes[1, 0].scatter(coords[:, 0], coords[:, 1], c=temps[:len(coords)], 
                                               cmap='hot', s=10, alpha=0.6)
                    axes[1, 0].set_xlabel('X (mm)')
                    axes[1, 0].set_ylabel('Y (mm)')
                    axes[1, 0].set_title(f'Temperature Spatial Distribution (XY Projection)\n{z_info}')
                    plt.colorbar(scatter, ax=axes[1, 0], label='Temperature (K)')
                    axes[1, 0].grid(True, alpha=0.3)
                
                axes[1, 1].scatter(sizes, temps, alpha=0.5, s=10)
                axes[1, 1].set_xlabel('Melt Pool Size (mm)')
                axes[1, 1].set_ylabel('Temperature (K)')
                axes[1, 1].set_title('Temperature vs Melt Pool Size')
                axes[1, 1].grid(True, alpha=0.3)
                
                plt.tight_layout()
                plt.show()
            else:
                display(HTML("<p style='color:orange;'>⚠️ Unexpected data format</p>"))
                
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))

def visualize_ct_scan():
    """Visualize CT scan data with 2D slice views."""
    with ct_viz_output:
        clear_output(wait=True)
        
        ct_data = query_results.get('ct_scan')
        
        if not ct_data:
            display(HTML("<p style='color:orange;'>⚠️ No CT scan data. Query CT data first.</p>"))
            return
        
        display(HTML("<p>🔄 Generating CT scan visualization...</p>"))
        
        try:
            from am_qadf.query.base_query_client import QueryResult
            
            if isinstance(ct_data, dict):
                voxel_grid = ct_data.get('voxel_grid', {})
                defects = ct_data.get('defect_locations', [])
                density_values = ct_data.get('density_values')
                
                if density_values is None and unified_client and mongo_client:
                    try:
                        from am_qadf.query import CTScanClient
                        ct_client = CTScanClient(mongo_client=mongo_client, use_mongodb=True)
                        model_id = query_results.get('model_id')
                        if model_id:
                            density_values = ct_client.get_density_values(model_id)
                    except Exception as e:
                        log_message(f"Could not load density from GridFS: {str(e)}", 'warning')
            elif isinstance(ct_data, QueryResult):
                if hasattr(ct_data, 'signals') and ct_data.signals:
                    density_values = ct_data.signals.get('density_values')
                else:
                    density_values = None
                voxel_grid = ct_data.metadata.get('voxel_grid', {}) if hasattr(ct_data, 'metadata') else {}
                defects = ct_data.metadata.get('defect_locations', []) if (hasattr(ct_data, 'metadata') and ct_data.metadata) else []
            else:
                display(HTML(f"<p style='color:orange;'>⚠️ Unexpected CT data type: {type(ct_data)}</p>"))
                return
            
            if density_values is None:
                display(HTML("<p style='color:orange;'>⚠️ Density values not available.</p>"))
                return
            
            dims = voxel_grid.get('dimensions', density_values.shape if hasattr(density_values, 'shape') else [100, 100, 100])
            
            fig, axes = plt.subplots(2, 2, figsize=(14, 10))
            
            z_slice = dims[2] // 2 if isinstance(dims, list) else density_values.shape[2] // 2
            im1 = axes[0, 0].imshow(density_values[:, :, z_slice], cmap='gray', origin='lower')
            axes[0, 0].set_title(f'XY Slice (Z = {z_slice})')
            plt.colorbar(im1, ax=axes[0, 0], label='Density')
            
            y_slice = dims[1] // 2 if isinstance(dims, list) else density_values.shape[1] // 2
            im2 = axes[0, 1].imshow(density_values[:, y_slice, :], cmap='gray', origin='lower')
            axes[0, 1].set_title(f'XZ Slice (Y = {y_slice})')
            plt.colorbar(im2, ax=axes[0, 1], label='Density')
            
            x_slice = dims[0] // 2 if isinstance(dims, list) else density_values.shape[0] // 2
            im3 = axes[1, 0].imshow(density_values[x_slice, :, :], cmap='gray', origin='lower')
            axes[1, 0].set_title(f'YZ Slice (X = {x_slice})')
            plt.colorbar(im3, ax=axes[1, 0], label='Density')
            
            if defects and len(defects) > 0:
                defect_coords_list = []
                for d in defects[:100]:
                    if isinstance(d, dict):
                        x = d.get('x', 0)
                        y = d.get('y', 0)
                        z = d.get('z', 0)
                    elif isinstance(d, (list, tuple)) and len(d) >= 3:
                        x = d[0]
                        y = d[1]
                        z = d[2]
                    else:
                        continue
                    defect_coords_list.append([x, y, z])
                
                if defect_coords_list:
                    defect_coords = np.array(defect_coords_list)
                    axes[1, 1].scatter(defect_coords[:, 0], defect_coords[:, 1], 
                                      c=defect_coords[:, 2], cmap='Reds', s=50, alpha=0.7)
                    axes[1, 1].set_xlabel('X (mm)')
                    axes[1, 1].set_ylabel('Y (mm)')
                    axes[1, 1].set_title(f'Defect Locations ({len(defects)} total)')
                    axes[1, 1].grid(True, alpha=0.3)
                else:
                    axes[1, 1].text(0.5, 0.5, 'No valid defect coordinates', 
                                   ha='center', va='center', transform=axes[1, 1].transAxes)
                    axes[1, 1].set_title('Defect Locations')
            else:
                axes[1, 1].text(0.5, 0.5, 'No defect data', 
                               ha='center', va='center', transform=axes[1, 1].transAxes)
                axes[1, 1].set_title('Defect Locations')
            
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            display(HTML(f"<p style='color:red;'>❌ Error: {str(e)}</p>"))
            import traceback
            display(HTML(f"<pre>{traceback.format_exc()}</pre>"))
            

def clear_results(button):
    """Clear all results and reset widgets."""
    global query_results, query_dataframes
    
    log_message("Clearing query results...", 'info')
    
    query_results = {}
    query_dataframes = {}
    
    with table_output:
        clear_output()
    with stats_output:
        clear_output()
    with viz_output:
        clear_output()
    with export_output:
        clear_output()
    
    # Clear logs
    with query_logs:
        clear_output(wait=True)
    
    log_message("Results cleared", 'success')
    
    status_text.value = "Ready to query"
    summary_text.value = "No query executed yet"
    update_status("Ready to query", 0)
    error_display.value = ""

def refresh_models(button):
    """Refresh the model list."""
    global models, model_options, operation_start_time
    
    # Initialize timing
    operation_start_time = time.time()
    
    # Clear logs
    with query_logs:
        clear_output(wait=True)
    
    log_message("Refreshing model list...", 'info')
    update_status("Refreshing models...", 0)
    
    if stl_client and mongo_client:
        try:
            models = stl_client.list_models(limit=100)
            model_options = [("━━━ Choose a model ━━━", None)]
            model_options.extend([
                (f"{m.get('filename', m.get('original_stem', m.get('model_name', 'Unknown')))} ({m.get('model_id', '')[:8]}...)", 
                 m.get('model_id')) for m in models
            ])
            if len(model_options) == 1:
                model_options.append(("No models available", None))
            model_dropdown.options = model_options
            log_message(f"Models refreshed: {len(models)} model(s) found", 'success')
            status_text.value = "<span style='color: green;'>✅ Models refreshed</span>"
            
            if operation_start_time:
                total_time = time.time() - operation_start_time
                log_message(f"Model refresh completed in {total_time:.2f}s", 'success')
            
            update_status("Models refreshed", 100)
        except Exception as e:
            log_message(f"Error refreshing models: {str(e)}", 'error')
            import traceback
            log_message(f"Traceback: {traceback.format_exc()}", 'error')
            status_text.value = f"<span style='color: red;'>Error: {str(e)}</span>"
            update_status("Error refreshing models", 0)
    else:
        log_message("MongoDB not available - using demo models", 'warning')
        status_text.value = "<span style='color: orange;'>⚠️ MongoDB not available - using demo models</span>"
        update_status("Using demo models", 0)



# Connect button events
query_button.on_click(execute_query)
clear_button.on_click(clear_results)
refresh_button.on_click(refresh_models)

# ============================================
# Main Layout
# ============================================

main_layout = VBox([
    top_panel,
    HBox([left_panel, center_panel, right_panel]),
    bottom_panel
])

# Display the interface
display(main_layout)


## Understanding QueryResult Structure

The `QueryResult` object contains the data returned from queries. Let's explore its structure:


# Display QueryResult structure information
structure_info = """
## QueryResult Structure

A `QueryResult` object contains:

### Attributes:
- **`data`**: Dictionary containing source-specific data
- **`metadata`**: Dictionary with query metadata (model_id, sources, filters, etc.)
- **`source`**: String indicating the data source
- **`spatial_bounds`**: Tuple of bounding box `((x_min, y_min, z_min), (x_max, y_max, z_max))`
- **`temporal_range`**: Tuple of time range `(start_time, end_time)`

### Data Structure by Source:

#### Hatching Data:
- **Points**: Array of (x, y, z) coordinates
- **Layers**: Layer information
- **Paths**: Hatching path data

#### Laser Parameters:
- **Points**: Array of (x, y, z) coordinates  
- **Signals**: Dictionary with 'power', 'speed', 'energy_density', etc.

#### CT Scan Data:
- **Points**: Array of (x, y, z) coordinates
- **Signals**: Dictionary with 'density', 'defect_flag', etc.

#### ISPM Monitoring:
- **Points**: Array of (x, y, z) coordinates
- **Signals**: Dictionary with 'temperature', 'sensor_id', 'timestamp', etc.

### Example Usage:

```python
# Query data
result = unified_client.query(
    model_id="my_model",
    sources=['hatching', 'laser'],
    spatial_bbox=((-50, -50, 0), (50, 50, 100)),
    temporal_range=(0, 1000)
)

# Access data
points = result.points  # Array of (x, y, z) coordinates
signals = result.signals  # Dictionary of signal arrays
metadata = result.metadata  # Query metadata
```
"""

display(Markdown(structure_info))


## Summary

Congratulations! You've learned how to query data from the AM-QADF warehouse.

### Key Takeaways

1. **UnifiedQueryClient**: Single interface for querying all data sources
2. **Spatial Filtering**: Filter data by bounding box or full model
3. **Temporal Filtering**: Filter by layers or time ranges
4. **Multi-Source Queries**: Query multiple sources in a single operation
5. **QueryResult Structure**: Understand the data structure returned from queries

### Next Steps

Proceed to:
- **02_Voxel_Grid_Creation.ipynb** - Learn to create voxel grids from queried data
- **03_Signal_Mapping_Fundamentals.ipynb** - Learn to map signals to voxel grids

### Related Resources

- Query Module Documentation: `../docs/AM_QADF/05-modules/query.md`
- API Reference: `../docs/AM_QADF/06-api-reference/query-api.md`
- Examples: `../examples/`
